In [20]:
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
import torch
import importlib
import torch.nn as nn

In [21]:
import pandas as pd
import numpy as np

INPUT_FILE = "data/processed/maleficent.csv"
df = pd.read_csv(INPUT_FILE)
df.head()

embeddings = []
for embedding in df['embedding']:
    temp = [float(x.strip(' []')) for x in embedding.split(',')]
    embeddings.append(temp)

categories = []
for category in df['category']:
    categories.append(category)

print("Vector size:\t\t\t", len(embeddings[0]))
print("Number of embeddings:\t\t", len(embeddings))
print("Number of category entries:\t", len(categories))

Vector size:			 384
Number of embeddings:		 1496
Number of category entries:	 1496


In [22]:
from siamese_network import SiameseNetwork

print(SiameseNetwork(128))

SiameseNetwork(
  (embedding): Sequential(
    (0): Linear(in_features=384, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=128, bias=True)
  )
)


In [23]:
from collections import Counter

data_dict = {}

for i, embedding in enumerate(embeddings):
    category = categories[i]

    if category not in data_dict:
        data_dict[category] = []

    # data_dict[category].append(list(map(float, embedding)))
    data_dict[category].append(embedding)


data_dict = {k: v for k, v in data_dict.items() if len(v) > 1}

In [24]:
print(len(data_dict['conversation']))
print(len(data_dict['jailbreak']))
print(len(data_dict['act_as']))

1402
70
24


In [25]:
from triplet_dataset import TripletDataset


siamese_dataset = TripletDataset(data_dict, False)

# Load the training dataset
num_workers = 4
batch_size = 1800
train_dataloader = DataLoader(
    siamese_dataset, shuffle=True, num_workers=num_workers, batch_size=batch_size
)

In [26]:
from torch import optim

embedding_dim = 128

siamese_net = SiameseNetwork(embedding_dim).cuda()
margin = 0.3
criterion = nn.TripletMarginLoss(margin=margin)

learning_rate = 0.0001
optimizer = optim.Adam(siamese_net.parameters(), lr=learning_rate)

n_epochs = 50

print(siamese_net)
print(f"Number of epochs: {n_epochs}")
print(f"Embedding dimension: {embedding_dim}")
print(f"Optimizer: Adam")
print(f"Learning rate: {learning_rate}")
print(f"Loss function: TripletMarginLoss")
print(f"Margin: {margin}")
print(f"Batch size: {batch_size}")
print(f"Number of workers: {num_workers}")
print(f"Dataset: {INPUT_FILE}")
print(f"Number of classes: {3}")
print(f"Number of samples: {len(siamese_dataset)}")

SiameseNetwork(
  (embedding): Sequential(
    (0): Linear(in_features=384, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=128, bias=True)
  )
)
Number of epochs: 50
Embedding dimension: 128
Optimizer: Adam
Learning rate: 0.0001
Loss function: TripletMarginLoss
Margin: 0.3
Batch size: 1800
Number of workers: 4
Dataset: data/processed/maleficent.csv
Number of classes: 3
Number of samples: 1496


In [27]:
def train(n_epochs, model, optimizer, criterion):
    global train_dataloader
    best_loss = float("inf")
    best_state = None
    iteration_number = 0
    counter = []
    loss_history = []

    for epoch in range(n_epochs):
        # Iterate over batches
        for i, (anchor, positive, negative) in enumerate(train_dataloader, 0):
            # Send the images to CUDA
            anchor, positive, negative = anchor.cuda(), positive.cuda(), negative.cuda()

            # Zero the gradients
            optimizer.zero_grad()

            # Pass in the anchor, positive, and negative images into the network
            output_anchor, output_positive, output_negative = model(anchor, positive, negative)

            # Compute the triplet loss
            loss_triplet = criterion(output_anchor, output_positive, output_negative)

            # If the validation loss is at a minimum
            if loss_triplet < best_loss:
                best_loss = loss_triplet
                best_state = model.state_dict()

            # Backpropagation
            loss_triplet.backward()

            # Optimize
            optimizer.step()

            # Every 10 batches print out the loss
            if i % 10 == 0:
                print(f"Epoch number {epoch}\n Current loss {loss_triplet.item()}\n")
                iteration_number += 10

                counter.append(iteration_number)
                loss_history.append(loss_triplet.item())

    print("Training finished")
    # print(counter)
    # print(loss_history)
    return counter, loss_history, best_state, best_loss

In [28]:
def show_plot(iteration, loss):
    plt.plot(iteration, loss)
    plt.show()

In [29]:
%%time
import time
from datetime import datetime

start_time = time.time()
# Training
counter, loss_history, best_state, best_loss = train(n_epochs, siamese_net, optimizer, criterion)
end_time = time.time()
execution_time = end_time - start_time
# Transforme seconds into minutes

minutes, seconds = divmod(execution_time, 60)

show_plot(counter, loss_history)



Epoch number 0
 Current loss 0.2805124819278717

Epoch number 1
 Current loss 0.26904165744781494

Epoch number 2
 Current loss 0.2616995871067047

Epoch number 3
 Current loss 0.2540184259414673

Epoch number 4
 Current loss 0.2438860386610031

Epoch number 5
 Current loss 0.22982053458690643

Epoch number 6
 Current loss 0.21692660450935364



In [ ]:
from datetime import datetime

now = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

# create folder
model_dir = f"trained/{now}"
os.makedirs(model_dir, exist_ok=True)
# Save the trained model
torch.save(siamese_net.state_dict(), os.path.join(model_dir, "state.pth"))

# Save the best state
torch.save(best_state, os.path.join(model_dir, f"best_state-{best_loss}.pth"))

torch.save(siamese_net, os.path.join(model_dir, "model.pth"))
with open(os.path.join(model_dir, "Architecture.txt"), "w") as f:
    f.write(str(siamese_net))
    f.write(f"\nExecution time: {int(minutes)} minutes and {seconds:.2f} seconds\n")
    f.write(f"Number of epochs: {n_epochs}\n")
    f.write(f"Embedding dimension: {embedding_dim}\n")
    f.write(f"Optimizer: Adam\n")
    f.write(f"Learning rate: {learning_rate}\n")
    f.write(f"Loss function: TripletMarginLoss\n")
    f.write(f"Margin: {margin}\n")
    f.write(f"Batch size: {batch_size}\n")
    f.write(f"Number of workers: {num_workers}\n")
    f.write(f"Dataset: {INPUT_FILE}\n")
    f.write(f"Number of classes: {len(csv_data)}\n")
    f.write(f"Number of samples: {len(siamese_dataset)}\n")

In [ ]:
print(best_loss)